In [1]:
library(caret, quiet = TRUE)
library(base64enc)
library(httr, quiet = TRUE)

library(mlbench)


Attaching package: ‘httr’

The following object is masked from ‘package:caret’:

    progress



# Build a Model

In [2]:

## multiclass classification in iris dataset:

data(BostonHousing)
BostonHousing$chas = as.numeric(BostonHousing$chas)

set.seed(1960)

dataset = BostonHousing[, -14] 

create_model  =  function() {

    model <- train(medv ~ ., data = BostonHousing, method = "nnet", linout=1, trace=FALSE)    

    return(model)
}


In [3]:
model = create_model()
# cat(model$feature_names)
# print(model)

Warning message in nominalTrainWorkflow(x = x, y = y, wts = weights, info = trainInfo, :
“There were missing values in resampled performance measures.”

In [4]:
pred_labels <- predict(model, BostonHousing[, -14] , type="raw")
df = data.frame(BostonHousing[,14])
names(df) = c("medv")
df$Estimator = pred_labels
df$Error = df$Estimator - df$medv
MAPE = mean(abs(df$Error / df$medv))
summary(df)
MAPE

      medv          Estimator.V1          Error.V1      
 Min.   : 5.00   Min.   :19.29224   Min.   :-16.907756  
 1st Qu.:17.02   1st Qu.:19.29224   1st Qu.: -3.507756  
 Median :21.20   Median :19.29224   Median : -0.065547  
 Mean   :22.53   Mean   :22.77502   Mean   :  0.242214  
 3rd Qu.:25.00   3rd Qu.:19.29251   3rd Qu.:  4.792244  
 Max.   :50.00   Max.   :35.07666   Max.   : 14.855700  

[1] 0.277937

# SQL Code Generation

In [5]:

test_ws_sql_gen = function(mod) {
    WS_URL = "https://sklearn2sql.herokuapp.com/model"
    WS_URL = "http://localhost:1888/model"
    model_serialized <- serialize(mod, NULL)
    b64_data = base64encode(model_serialized)
    data = list(Name = "xgboost_test_model", SerializedModel = b64_data , SQLDialect = "postgresql" , Mode="caret")
    r = POST(WS_URL, body = data, encode = "json")
    # print(r)
    content = content(r)
    # print(content)
    lSQL = content$model$SQLGenrationResult[[1]]$SQL # content["model"]["SQLGenrationResult"][0]["SQL"]
    return(lSQL);
}

In [6]:
lModelSQL = test_ws_sql_gen(model)
cat(lModelSQL)


WITH "IL" AS 
(SELECT "ADS"."KEY" AS "KEY", CAST("ADS"."Feature_0" AS FLOAT) AS "Feature_0", CAST("ADS"."Feature_1" AS FLOAT) AS "Feature_1", CAST("ADS"."Feature_2" AS FLOAT) AS "Feature_2", CAST("ADS"."Feature_3" AS FLOAT) AS "Feature_3", CAST("ADS"."Feature_4" AS FLOAT) AS "Feature_4", CAST("ADS"."Feature_5" AS FLOAT) AS "Feature_5", CAST("ADS"."Feature_6" AS FLOAT) AS "Feature_6", CAST("ADS"."Feature_7" AS FLOAT) AS "Feature_7", CAST("ADS"."Feature_8" AS FLOAT) AS "Feature_8", CAST("ADS"."Feature_9" AS FLOAT) AS "Feature_9", CAST("ADS"."Feature_10" AS FLOAT) AS "Feature_10", CAST("ADS"."Feature_11" AS FLOAT) AS "Feature_11", CAST("ADS"."Feature_12" AS FLOAT) AS "Feature_12" 
FROM "INPUT_DATA" AS "ADS"), 
"HL_BA_1" AS 
(SELECT "IL"."KEY" AS "KEY", 0.7397418 * "IL"."Feature_0" + 0.9450092 * "IL"."Feature_1" + 1.466952 * "IL"."Feature_2" + 2.892708 * "IL"."Feature_3" + -0.3298537 * "IL"."Feature_4" + 10.14366 * "IL"."Feature_5" + 0.3438722 * "IL"."Feature_6" + -11.1901 * "IL"."Feature_

# Execute the SQL Code

In [7]:
library(RODBC)
conn = odbcConnect("pgsql", uid="db", pwd="db", case="nochange")
odbcSetAutoCommit(conn , autoCommit = TRUE)

[1] 0

In [8]:
df_sql = dataset
names(df_sql) = sprintf("Feature_%d",0:(ncol(df_sql)-1))
df_sql$KEY = seq.int(nrow(dataset))

sqlDrop(conn , "INPUT_DATA" , errors = FALSE)
sqlSave(conn, df_sql, tablename = "INPUT_DATA", verbose = FALSE)

# df_sql

In [9]:
colnames(df_sql)
# odbcGetInfo(conn)
# sqlTables(conn)

[1] "Feature_0"  "Feature_1"  "Feature_2"  "Feature_3"  "Feature_4" 
 [6] "Feature_5"  "Feature_6"  "Feature_7"  "Feature_8"  "Feature_9" 
[11] "Feature_10" "Feature_11" "Feature_12" "KEY"

In [10]:
df_sql_out = sqlQuery(conn, lModelSQL)
head(df_sql_out[order(df_sql_out$KEY),])

KEY,Estimator
1,35.07664
2,19.29224
3,35.07666
4,35.07666
5,19.29296
6,19.29225


In [11]:
# df_sql_out

# R NNET Output

In [12]:
estimator  =  predict(model, dataset, type = "raw")
df_r_out = data.frame(estimator)
names(df_r_out) = c("Estimator")

df_r_out$KEY = seq.int(nrow(dataset))
head(df_r_out)


Estimator,KEY
35.07664,1
19.29224,2
35.07666,3
35.07666,4
19.29296,5
19.29225,6


# Compare R and SQL output

In [13]:
df_merge = merge(x = df_r_out, y = df_sql_out, by = "KEY", all = TRUE, , suffixes = c("_1","_2"))
head(df_merge)

KEY,Estimator_1,Estimator_2
1,35.07664,35.07664
2,19.29224,19.29224
3,35.07666,35.07666
4,35.07666,35.07666
5,19.29296,19.29296
6,19.29225,19.29225


In [14]:
df_merge$Error = df_merge$Estimator_1 - df_merge$Estimator_2
df_merge$AbsError = abs(df_merge$Error)
head(df_merge)


KEY,Estimator_1,Estimator_2,Error,AbsError
1,35.07664,35.07664,9.442187e-07,9.442187e-07
2,19.29224,19.29224,4.175340e-06,4.175340e-06
3,35.07666,35.07666,9.446836e-07,9.446836e-07
4,35.07666,35.07666,9.542851e-07,9.542851e-07
5,19.29296,19.29296,4.179421e-06,4.179421e-06
6,19.29225,19.29225,4.175385e-06,4.175385e-06


In [15]:
df_merge_largest_errors = df_merge[df_merge$AbsError > 0.0001,]
df_merge_largest_errors

KEY,Estimator_1,Estimator_2,Error,AbsError


In [16]:
stopifnot(nrow(df_merge_largest_errors) == 0)

In [17]:
summary(df_sql_out)

      KEY          Estimator    
 Min.   :  1.0   Min.   :19.29  
 1st Qu.:127.2   1st Qu.:19.29  
 Median :253.5   Median :19.29  
 Mean   :253.5   Mean   :22.78  
 3rd Qu.:379.8   3rd Qu.:19.29  
 Max.   :506.0   Max.   :35.08  

In [18]:
summary(df_r_out)

   Estimator          KEY       
 Min.   :19.29   Min.   :  1.0  
 1st Qu.:19.29   1st Qu.:127.2  
 Median :19.29   Median :253.5  
 Mean   :22.78   Mean   :253.5  
 3rd Qu.:19.29   3rd Qu.:379.8  
 Max.   :35.08   Max.   :506.0  

In [19]:
summary(df_merge)

      KEY         Estimator_1     Estimator_2        Error          
 Min.   :  1.0   Min.   :19.29   Min.   :19.29   Min.   :8.789e-07  
 1st Qu.:127.2   1st Qu.:19.29   1st Qu.:19.29   1st Qu.:4.174e-06  
 Median :253.5   Median :19.29   Median :19.29   Median :4.175e-06  
 Mean   :253.5   Mean   :22.78   Mean   :22.78   Mean   :4.098e-06  
 3rd Qu.:379.8   3rd Qu.:19.29   3rd Qu.:19.29   3rd Qu.:4.175e-06  
 Max.   :506.0   Max.   :35.08   Max.   :35.08   Max.   :8.059e-05  
    AbsError        
 Min.   :8.789e-07  
 1st Qu.:4.174e-06  
 Median :4.175e-06  
 Mean   :4.098e-06  
 3rd Qu.:4.175e-06  
 Max.   :8.059e-05  